In [2]:
#importing openslide:
OPENSLIDE_PATH = r"C:\Users\Kevin\Downloads\openslide-win64-20221217\bin"

import os
if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

In [2]:
import numpy as np
from PIL import Image
import math
import os
import pandas as pd
from tqdm import tqdm

In [3]:
# First get list of all deparafinized and student_score >2 and CLUE:
all_df_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\wsi_list_230314.xlsx'
all_df = pd.read_excel(all_df_src)
desired_df_name = all_df[all_df.scan_batch == 3] # CLUE
desired_df_name1 = all_df[all_df.scan_batch == 4] # CLUE
desired_name = list(desired_df_name.wsi_name)
desired_name1 = list(desired_df_name1.wsi_name)
desired_names = desired_name + desired_name1
he_desired_names = [x for x in desired_names if x[-3:] == "_he"]
he_desired_names_src = [x+".ndpi" for x in he_desired_names]
us_desired_names = [x for x in desired_names if not x[-3:] == "_he"]
us_desired_names_src = [x+".ndpi" for x in us_desired_names]
US_dst_src = r'\\shelter\Kyu\unstain2stain\tiles\unregistered_tiles\US'
US_ndpi_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\Unstained'
US_desired_src = [os.path.join(US_ndpi_src,x) for x in us_desired_names_src]
US_desired_src

In [ ]:
for idx, src in tqdm(enumerate(US_desired_src),total=len(US_desired_src),colour='red',desc='WSI Processed'):
    wsi_name = us_desired_names[idx]
    save_src = os.path.join(US_dst_src,wsi_name)
    if not os.path.exists(save_src):
        os.makedirs(save_src)
    slide = openslide.open_slide(src)
    target_level = 0
    target_dim = slide.level_dimensions[target_level]
    width, height = target_dim
    # unpad since it's white background anyways
    tiles_x_unpad = int((width % 1024) / 2)
    tiles_y_unpad = int((height % 1024) / 2)
    # read from 0+tiles_x_unpad/y_pad and max-tiles_x_unpad/ypad:
    num_rows= ((width - tiles_x_unpad) - (0 + tiles_x_unpad)) / 1024
    num_cols= ((height - tiles_y_unpad) - (0 + tiles_y_unpad)) / 1024
    num_rows = int(num_rows)
    num_cols = int(num_cols)
    print(num_rows)
    print(num_cols)
        # tile to 1024 x 1024
    for row_idx in range(num_rows): # 0~num_rows-1
        for col_idx in range(num_cols): # 0~num_cols-1
            xcoord= tiles_x_unpad + 1024 * row_idx
            ycoord= tiles_y_unpad + 1024 * col_idx
            nsimage = slide.read_region(location=(xcoord,ycoord),level=target_level,size=(1024,1024))
            nsimage = np.array(nsimage)
            imgidx = row_idx * num_cols + (col_idx + 1)
            imgidx = f"{imgidx:04d}"
            imgcoord = str(xcoord) + "_" + str(ycoord) + "xy"
            imgname = imgcoord + imgidx + ".png"
            tile_save_src = os.path.join(save_src,imgname)
            Image.fromarray(nsimage).save(tile_save_src)

### Parallel processing:

In [1]:
import numpy as np
from PIL import Image
import math
import os
import pandas as pd
from tqdm import tqdm
import pathos
from pathos.multiprocessing import ProcessPool
num_nodes = pathos.helpers.cpu_count()

In [2]:
# First get list of all deparafinized and student_score >2 and CLUE:
all_df_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\wsi_list_230314.xlsx'
all_df = pd.read_excel(all_df_src)
desired_df_name = all_df[all_df.scan_batch == 3] # CLUE
desired_df_name1 = all_df[all_df.scan_batch == 4] # CLUE
desired_name = list(desired_df_name.wsi_name)
desired_name1 = list(desired_df_name1.wsi_name)
desired_names = desired_name + desired_name1
he_desired_names = [x for x in desired_names if x[-3:] == "_he"]
he_desired_names_src = [x+".ndpi" for x in he_desired_names]
us_desired_names = [x for x in desired_names if not x[-3:] == "_he"]
us_desired_names_src = [x+".ndpi" for x in us_desired_names]

US_dst_src = r'\\shelter\Kyu\unstain2stain\tiles\unregistered_tiles\US'
US_ndpi_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\Unstained'
US_desired_src = [os.path.join(US_ndpi_src,x) for x in us_desired_names_src]
US_desired_src

['\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_1.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_2.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_3.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_4.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_5.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_6.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_7.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_8.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14684_9.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14832_1.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14832_2.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14832_3

In [4]:
# code to select only the US_desired_src of the ones that were somehow not generated properly:
# Unregistered tiles:
# Problems with, so retile:
# HE: OTS_14684_4_he, OTS_14832_3_he, OTS_14832_4_he, OTS_14832_7_he
# US: OTS_14832_2, OTS_14832_3, OTS_14832_4,  all the way to 16.
US_desired_src = US_desired_src[-15:]
US_desired_src

['\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14832_2.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14832_3.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14832_4.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14832_5.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14832_6.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14832_7.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14832_8.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14832_9.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14832_10.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14832_11.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_14832_12.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\Unstained\\OTS_1483

In [5]:
#importing openslide:
OPENSLIDE_PATH = r"C:\Users\labuser1\Downloads\openslide-win64-20230414\bin"

import os
if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

In [6]:
def tile_wsi(src, wsi_name, save_src):
    import os
    import numpy as np
    from PIL import Image
    import math
    import pandas as pd
    from tqdm import tqdm
    import time
    from pathos.multiprocessing import ProcessPool
    OPENSLIDE_PATH = r"C:\Users\labuser1\Downloads\openslide-win64-20230414\bin"
    if hasattr(os, 'add_dll_directory'):
        # Python >= 3.8 on Windows
        with os.add_dll_directory(OPENSLIDE_PATH):
            import openslide
    else:
        import openslide
    if not os.path.exists(save_src):
        os.makedirs(save_src)
    slide = openslide.open_slide(src)
    target_level = 0
    target_dim = slide.level_dimensions[target_level]
    width, height = target_dim
    # unpad since it's white background anyways
    tiles_x_unpad = int((width % 1024) / 2)
    tiles_y_unpad = int((height % 1024) / 2)
    # read from 0+tiles_x_unpad/y_pad and max-tiles_x_unpad/ypad:
    num_rows= ((width - tiles_x_unpad) - (0 + tiles_x_unpad)) / 1024
    num_cols= ((height - tiles_y_unpad) - (0 + tiles_y_unpad)) / 1024
    num_rows = int(num_rows)
    num_cols = int(num_cols)
    # tile to 1024 x 1024
    for row_idx in range(num_rows): # 0~num_rows-1
        for col_idx in range(num_cols): # 0~num_cols-1
            xcoord= tiles_x_unpad + 1024 * row_idx
            ycoord= tiles_y_unpad + 1024 * col_idx
            nsimage = slide.read_region(location=(xcoord,ycoord),level=target_level,size=(1024,1024))
            nsimage = np.array(nsimage)
            imgidx = row_idx * num_cols + (col_idx + 1)
            imgidx = f"{imgidx:04d}"
            imgcoord = str(xcoord) + "_" + str(ycoord) + "xy"
            imgname = imgcoord + imgidx + ".png"
            tile_save_src = os.path.join(save_src, imgname)
            Image.fromarray(nsimage).save(tile_save_src)

In [ ]:
# for US:
import time
start = time.time()
pool = ProcessPool(nodes=num_nodes)
results = []

for idx, src in tqdm(enumerate(US_desired_src), total=len(US_desired_src), colour='red', desc='WSI Processed'):
    wsi_name = us_desired_names[idx]
    save_src = os.path.join(US_dst_src, wsi_name)
    results.append(pool.apipe(tile_wsi, src, wsi_name, save_src))

# Get the results
for result in results:
    import os
    result.get()

# Close the process pool
pool.close()

end = time.time()
print("time it took to tile all US images: {} hours".format((start-end)/3600))

WSI Processed: 100%|██████████| 15/15 [00:00<?, ?it/s]


### Same code for HE:

In [9]:
import numpy as np
from PIL import Image
import math
import os
import pandas as pd
from tqdm import tqdm
import pathos
from pathos.multiprocessing import ProcessPool
num_nodes = pathos.helpers.cpu_count()
OPENSLIDE_PATH = r"C:\Users\labuser1\Downloads\openslide-win64-20230414\bin" # this will change
if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

In [10]:
# First get list of all deparafinized and student_score >2 and CLUE:
all_df_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\wsi_list_230314.xlsx'
all_df = pd.read_excel(all_df_src)
desired_df_name = all_df[all_df.scan_batch == 3] # CLUE
desired_df_name1 = all_df[all_df.scan_batch == 4] # CLUE
desired_name = list(desired_df_name.wsi_name)
desired_name1 = list(desired_df_name1.wsi_name)
desired_names = desired_name + desired_name1
he_desired_names = [x for x in desired_names if x[-3:] == "_he"]
he_desired_names_src = [x+".ndpi" for x in he_desired_names]
us_desired_names = [x for x in desired_names if not x[-3:] == "_he"]
us_desired_names_src = [x+".ndpi" for x in us_desired_names]

HE_dst_src = r'\\shelter\Kyu\unstain2stain\tiles\unregistered_tiles\HE'
HE_ndpi_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE'
HE_desired_src = [os.path.join(HE_ndpi_src,x) for x in he_desired_names_src]
HE_desired_src

['\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14684_1_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14684_2_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14684_3_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14684_4_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14684_5_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14684_6_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14684_7_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14684_8_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14684_9_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14832_1_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14832_2_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14832_3_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unsta

In [11]:
# code to select only the US_desired_src of the ones that were somehow not generated properly:
# Unregistered tiles:
# Problems with, so retile:
# HE: OTS_14684_4_he, OTS_14832_3_he, OTS_14832_4_he, OTS_14832_7_he
# US: OTS_14832_2, OTS_14832_3, OTS_14832_4,  all the way to 16.
tmp = []
tmp.append(HE_desired_src[15])
tmp.append(HE_desired_src[12])
tmp.append(HE_desired_src[11])
tmp.append(HE_desired_src[3])
HE_desired_src = tmp

In [12]:
HE_desired_src

['\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14832_7_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14832_4_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14832_3_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14684_4_he.ndpi']

In [15]:
def tile_wsi(src, wsi_name, save_src):
    import os
    import numpy as np
    from PIL import Image
    import time
    import math
    import pandas as pd
    import time
    from tqdm import tqdm
    from pathos.multiprocessing import ProcessPool
    OPENSLIDE_PATH = r"C:\Users\labuser1\Downloads\openslide-win64-20230414\bin" # this will change
    if hasattr(os, 'add_dll_directory'):
        # Python >= 3.8 on Windows
        with os.add_dll_directory(OPENSLIDE_PATH):
            import openslide
    else:
        import openslide
    if not os.path.exists(save_src):
        os.makedirs(save_src)
    slide = openslide.open_slide(src)
    target_level = 0
    target_dim = slide.level_dimensions[target_level]
    width, height = target_dim
    # unpad since it's white background anyways
    tiles_x_unpad = int((width % 1024) / 2)
    tiles_y_unpad = int((height % 1024) / 2)
    # read from 0+tiles_x_unpad/y_pad and max-tiles_x_unpad/ypad:
    num_rows= ((width - tiles_x_unpad) - (0 + tiles_x_unpad)) / 1024
    num_cols= ((height - tiles_y_unpad) - (0 + tiles_y_unpad)) / 1024
    num_rows = int(num_rows)
    num_cols = int(num_cols)
    # tile to 1024 x 1024
    for row_idx in range(num_rows): # 0~num_rows-1
        for col_idx in range(num_cols): # 0~num_cols-1
            xcoord= tiles_x_unpad + 1024 * row_idx
            ycoord= tiles_y_unpad + 1024 * col_idx
            try:
                heimage = slide.read_region(location=(xcoord,ycoord),level=target_level,size=(1024,1024))
            except:
                print("Corrupted Data, cannot tile image for coordinates {} {}".format(xcoord,ycoord))
                continue
            heimage = np.array(heimage)
            imgidx = row_idx * num_cols + (col_idx + 1)
            imgidx = f"{imgidx:04d}"
            imgcoord = str(xcoord) + "_" + str(ycoord) + "xy"
            imgname = imgcoord + imgidx + ".png"
            tile_save_src = os.path.join(save_src, imgname)
            Image.fromarray(heimage).save(tile_save_src)

In [16]:
# for HE:
import time
OPENSLIDE_PATH = r"C:\Users\labuser1\Downloads\openslide-win64-20230414\bin" # this will change
if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide
start = time.time()
pool = ProcessPool(nodes=num_nodes) #pathos.helpers.cpu_count()
results = []

for idx, src in tqdm(enumerate(HE_desired_src), total=len(HE_desired_src), colour='red', desc='WSI Processed'):
    OPENSLIDE_PATH = r"C:\Users\labuser1\Downloads\openslide-win64-20230414\bin" # this will change
    if hasattr(os, 'add_dll_directory'):
        # Python >= 3.8 on Windows
        with os.add_dll_directory(OPENSLIDE_PATH):
            import openslide
    else:
        import openslide
    wsi_name = HE_desired_src[idx].split('HE')[1].split('\\')[1].split('.ndpi')[0]
    save_src = os.path.join(HE_dst_src, wsi_name)
    results.append(pool.apipe(tile_wsi, src, wsi_name, save_src))

# Get the results
for result in results:
    import os
    if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
        with os.add_dll_directory(OPENSLIDE_PATH):
            import openslide
    else:
        import openslide
    result.get()

# Close the process pool
pool.close()
end = time.time()
print("time it took to tile all US images: {} hours".format((end-start)/3600))

WSI Processed: 100%|██████████| 4/4 [00:00<?, ?it/s]


time it took to tile all US images: -3.1104557132720947 hours


### Still some corrupted images with openslide, maybe try tiling with no parallel processing? For HE (only unfinished corrupted images during parallel processing:)


In [1]:
import numpy as np
from PIL import Image
import math
import os
import pandas as pd
from tqdm import tqdm

OPENSLIDE_PATH = r"C:\Users\labuser1\Downloads\openslide-win64-20230414\bin"  # this will change
if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide
# First get list of all deparafinized and student_score >2 and CLUE:
all_df_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\wsi_list_230314.xlsx'
all_df = pd.read_excel(all_df_src)
desired_df_name = all_df[all_df.scan_batch == 3]  # CLUE
desired_df_name1 = all_df[all_df.scan_batch == 4]  # CLUE
desired_name = list(desired_df_name.wsi_name)
desired_name1 = list(desired_df_name1.wsi_name)
desired_names = desired_name + desired_name1
he_desired_names = [x for x in desired_names if x[-3:] == "_he"]
he_desired_names_src = [x + ".ndpi" for x in he_desired_names]
us_desired_names = [x for x in desired_names if not x[-3:] == "_he"]
us_desired_names_src = [x + ".ndpi" for x in us_desired_names]

HE_dst_src = r'\\shelter\Kyu\unstain2stain\tiles\unregistered_tiles\HE'
HE_ndpi_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE'
HE_desired_src = [os.path.join(HE_ndpi_src, x) for x in he_desired_names_src]
HE_desired_src
# code to select only the US_desired_src of the ones that were somehow not generated properly:
# Unregistered tiles:
# Problems with, so retile:
# HE: OTS_14684_4_he, OTS_14832_3_he, OTS_14832_4_he, OTS_14832_7_he
# US: OTS_14832_2, OTS_14832_3, OTS_14832_4,  all the way to 16.
tmp = []
tmp.append(HE_desired_src[15])
tmp.append(HE_desired_src[12])
tmp.append(HE_desired_src[11])
tmp.append(HE_desired_src[3])
HE_desired_src = tmp
HE_desired_src

['\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14832_7_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14832_4_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14832_3_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14684_4_he.ndpi']

In [3]:
for idx, src in tqdm(enumerate(HE_desired_src),total=len(HE_desired_src),colour='red',desc='WSI Processed'):
    wsi_name = src.split('HE')[1].split('\\')[1].split('.ndpi')[0]
    save_src = os.path.join(HE_dst_src,wsi_name)
    if not os.path.exists(save_src):
        os.makedirs(save_src)
    slide = openslide.open_slide(src)
    target_level = 0
    target_dim = slide.level_dimensions[target_level]
    width, height = target_dim
    # unpad since it's white background anyways
    tiles_x_unpad = int((width % 1024) / 2)
    tiles_y_unpad = int((height % 1024) / 2)
    # read from 0+tiles_x_unpad/y_pad and max-tiles_x_unpad/ypad:
    num_rows= ((width - tiles_x_unpad) - tiles_x_unpad) / 1024
    num_cols= ((height - tiles_y_unpad) - tiles_y_unpad) / 1024
    num_rows = int(num_rows)
    num_cols = int(num_cols)
    print("num rows is {}".format(num_rows))
    print("num cols is {}".format(num_cols))
    # tile to 1024 x 1024
    for row_idx in tqdm(range(num_rows),colour='red',desc='numrows'): # 0~num_rows-1
        for col_idx in tqdm(range(num_cols),colour='red',desc='numcols'): # 0~num_cols-1
            xcoord= tiles_x_unpad + 1024 * row_idx
            ycoord= tiles_y_unpad + 1024 * col_idx
            heimage = slide.read_region(location=(xcoord,ycoord),level=target_level,size=(1024,1024))
            heimage = np.array(heimage)
            imgidx = row_idx * num_cols + (col_idx + 1)
            imgidx = f"{imgidx:04d}"
            imgcoord = str(xcoord) + "_" + str(ycoord) + "xy"
            imgname = imgcoord + imgidx + ".png"
            tile_save_src = os.path.join(save_src,imgname)
            Image.fromarray(heimage).save(tile_save_src)

WSI Processed:   0%|          | 0/4 [00:00<?, ?it/s]

num rows is 135
num cols is 77



numrows:   0%|          | 0/135 [00:00<?, ?it/s]

numcols:   0%|          | 0/77 [00:00<?, ?it/s]

numcols:   1%|▏         | 1/77 [00:01<02:11,  1.73s/it]

numcols:   3%|▎         | 2/77 [00:02<01:42,  1.36s/it]

numcols:   4%|▍         | 3/77 [00:03<01:30,  1.23s/it]

numcols:   5%|▌         | 4/77 [00:04<01:24,  1.16s/it]

numcols:   6%|▋         | 5/77 [00:06<01:35,  1.32s/it]

numcols:   8%|▊         | 6/77 [00:07<01:27,  1.23s/it]

numcols:   9%|▉         | 7/77 [00:08<01:22,  1.18s/it]

numcols:  10%|█         | 8/77 [00:09<01:18,  1.14s/it]

numcols:  12%|█▏        | 9/77 [00:10<01:15,  1.12s/it]

numcols:  13%|█▎        | 10/77 [00:12<01:21,  1.21s/it]

numcols:  14%|█▍        | 11/77 [00:13<01:17,  1.17s/it]

numcols:  16%|█▌        | 12/77 [00:14<01:13,  1.14s/it]

numcols:  17%|█▋        | 13/77 [00:15<01:12,  1.13s/it]

numcols:  18%|█▊        | 14/77 [00:16<01:09,  1.11s/it]

numcols:  19%|█▉        | 15/77 [00:17<01:14,  1.20s/it]

numcols:  21%|██        | 16/77 [00:18<

### Try one at a time:

In [1]:
import numpy as np
from PIL import Image
import math
import os
import pandas as pd
from tqdm import tqdm

OPENSLIDE_PATH = r"C:\Users\labuser1\Downloads\openslide-win64-20230414\bin"  # this will change
if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide
# First get list of all deparafinized and student_score >2 and CLUE:
all_df_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\wsi_list_230314.xlsx'
all_df = pd.read_excel(all_df_src)
desired_df_name = all_df[all_df.scan_batch == 3]  # CLUE
desired_df_name1 = all_df[all_df.scan_batch == 4]  # CLUE
desired_name = list(desired_df_name.wsi_name)
desired_name1 = list(desired_df_name1.wsi_name)
desired_names = desired_name + desired_name1
he_desired_names = [x for x in desired_names if x[-3:] == "_he"]
he_desired_names_src = [x + ".ndpi" for x in he_desired_names]
us_desired_names = [x for x in desired_names if not x[-3:] == "_he"]
us_desired_names_src = [x + ".ndpi" for x in us_desired_names]

HE_dst_src = r'\\shelter\Kyu\unstain2stain\tiles\unregistered_tiles\HE'
HE_ndpi_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\HE'
HE_desired_src = [os.path.join(HE_ndpi_src, x) for x in he_desired_names_src]
HE_desired_src
# code to select only the US_desired_src of the ones that were somehow not generated properly:
# Unregistered tiles:
# Problems with, so retile:
# HE: OTS_14684_4_he, OTS_14832_3_he, OTS_14832_4_he, OTS_14832_7_he
# US: OTS_14832_2, OTS_14832_3, OTS_14832_4,  all the way to 16.
tmp = []
tmp.append(HE_desired_src[15])
tmp.append(HE_desired_src[12])
tmp.append(HE_desired_src[11])
tmp.append(HE_desired_src[3])
HE_desired_src = tmp
HE_desired_src

['\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14832_7_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14832_4_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14832_3_he.ndpi',
 '\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14684_4_he.ndpi']

In [4]:
slide

OpenSlide('\\\\shelter\\Kyu\\unstain2stain\\unstain2stain_wsi\\HE\\OTS_14832_7_he.ndpi')

In [5]:
src = HE_desired_src[0]
wsi_name = src.split('HE')[1].split('\\')[1].split('.ndpi')[0]
save_src = os.path.join(HE_dst_src,wsi_name)
if not os.path.exists(save_src):
    os.makedirs(save_src)
slide = openslide.open_slide(src)

target_level = 0
target_dim = slide.level_dimensions[target_level]
width, height = target_dim
wsi_img_ra = np.array(slide.read_region(location=(0,0),level=target_level,size=(target_dim)))
# unpad since it's white background anyways
tiles_x_unpad = int((width % 1024) / 2)
tiles_y_unpad = int((height % 1024) / 2)
# read from 0+tiles_x_unpad/y_pad and max-tiles_x_unpad/ypad:
num_rows= ((width - tiles_x_unpad) - tiles_x_unpad) / 1024
num_cols= ((height - tiles_y_unpad) - tiles_y_unpad) / 1024
num_rows = int(num_rows)
num_cols = int(num_cols)
print("num rows is {}".format(num_rows))
print("num cols is {}".format(num_cols))
# tile to 1024 x 1024
for row_idx in tqdm(range(num_rows),colour='red',desc='numrows'): # 0~num_rows-1
    for col_idx in tqdm(range(num_cols),colour='red',desc='numcols'): # 0~num_cols-1
        xcoord= tiles_x_unpad + 1024 * row_idx
        ycoord= tiles_y_unpad + 1024 * col_idx
        heimage = wsi_img_ra[ycoord:ycoord+1024,xcoord:xcoord+1024]
        # try:
        #     heimage = slide.read_region(location=(xcoord,ycoord),level=target_level,size=(1024,1024))
        # except:
        #     print("Image file corrupted for image {} with x,y coord {} {}".format(wsi_name, xcoord, ycoord))
        #     break
        imgidx = row_idx * num_cols + (col_idx + 1)
        imgidx = f"{imgidx:04d}"
        imgcoord = str(xcoord) + "_" + str(ycoord) + "xy"
        imgname = imgcoord + imgidx + ".png"
        tile_save_src = os.path.join(save_src,imgname)
        Image.fromarray(heimage).save(tile_save_src)

OpenSlideError: Corrupt JPEG data: premature end of data segment

### Still some corrupted images with openslide, maybe try tiling with no parallel processing? For HE (only unfinished corrupted images during parallel processing:)
### For US:

In [ ]:
import numpy as np
from PIL import Image
import math
import os
import pandas as pd
from tqdm import tqdm
OPENSLIDE_PATH = r"C:\Users\labuser1\Downloads\openslide-win64-20230414\bin"
if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
else:
    import openslide

# First get list of all deparafinized and student_score >2 and CLUE:
all_df_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\wsi_list_230314.xlsx'
all_df = pd.read_excel(all_df_src)
desired_df_name = all_df[all_df.scan_batch == 3]  # CLUE
desired_df_name1 = all_df[all_df.scan_batch == 4]  # CLUE
desired_name = list(desired_df_name.wsi_name)
desired_name1 = list(desired_df_name1.wsi_name)
desired_names = desired_name + desired_name1
us_desired_names = [x for x in desired_names if not x[-3:] == "_he"]
us_desired_names_src = [x + ".ndpi" for x in us_desired_names]

US_dst_src = r'\\shelter\Kyu\unstain2stain\tiles\unregistered_tiles\US'
US_ndpi_src = r'\\shelter\Kyu\unstain2stain\unstain2stain_wsi\Unstained'
US_desired_src = [os.path.join(US_ndpi_src, x) for x in us_desired_names_src]
US_desired_src
# code to select only the US_desired_src of the ones that were somehow not generated properly:
# Unregistered tiles:
# Problems with, so retile:
# HE: OTS_14684_4_he, OTS_14832_3_he, OTS_14832_4_he, OTS_14832_7_he
# US: OTS_14832_2, OTS_14832_3, OTS_14832_4,  all the way to 16.
US_desired_src = US_desired_src[-15:]
US_desired_src

In [ ]:
for idx, src in tqdm(enumerate(US_desired_src),total=len(US_desired_src),colour='red',desc='WSI Processed'):
    wsi_name = src.split('US')[1].split('\\')[1].split('.ndpi')[0]
    save_src = os.path.join(US_dst_src,wsi_name)
    if not os.path.exists(save_src):
        os.makedirs(save_src)
    slide = openslide.open_slide(src)
    target_level = 0
    target_dim = slide.level_dimensions[target_level]
    width, height = target_dim
    # unpad since it's white background anyways
    tiles_x_unpad = int((width % 1024) / 2)
    tiles_y_unpad = int((height % 1024) / 2)
    # read from 0+tiles_x_unpad/y_pad and max-tiles_x_unpad/ypad:
    num_rows= ((width - tiles_x_unpad) - tiles_x_unpad) / 1024
    num_cols= ((height - tiles_y_unpad) - tiles_y_unpad) / 1024
    num_rows = int(num_rows)
    num_cols = int(num_cols)
    print("num rows is {}".format(num_rows))
    print("num cols is {}".format(num_cols))
    # tile to 1024 x 1024
    for row_idx in tqdm(range(num_rows),colour='red',desc='numrows'): # 0~num_rows-1
        for col_idx in tqdm(range(num_cols),colour='red',desc='numcols'): # 0~num_cols-1
            xcoord= tiles_x_unpad + 1024 * row_idx
            ycoord= tiles_y_unpad + 1024 * col_idx
            nsimage = slide.read_region(location=(xcoord,ycoord),level=target_level,size=(1024,1024))
            nsimage = np.array(nsimage)
            imgidx = row_idx * num_cols + (col_idx + 1)
            imgidx = f"{imgidx:04d}"
            imgcoord = str(xcoord) + "_" + str(ycoord) + "xy"
            imgname = imgcoord + imgidx + ".png"
            tile_save_src = os.path.join(save_src,imgname)
            Image.fromarray(nsimage).save(tile_save_src)

In [2]:
from PIL import Image
import os

# Set the input and output directories
input_dir = r"\\shelter\Kyu\unstain2stain\tiles\registered_tiles\HE_train_val_test_split\fold_A\train"
output_dir = r"\\shelter\Kyu\unstain2stain\tiles\registered_tiles\HE_train_val_test_split\fold_A\train\tile_512"

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Loop through all the PNG files in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith(".png"):
        # Open the image
        image = Image.open(os.path.join(input_dir, filename))

        # Get the width and height of the image
        width, height = image.size

        # Check that the image is 1024x1024 pixels
        if width == 1024 and height == 1024:
            # Tile the image into 16 256x256 tiles
            for i in range(2):
                for j in range(2):
                    x = i * 512
                    y = j * 512
                    tile = image.crop((x, y, x + 512, y + 512))

                    # Save the tile with a unique name
                    tile_filename = f"{os.path.splitext(filename)[0]}_{i}_{j}.png"
                    tile.save(os.path.join(output_dir, tile_filename))


In [ ]:
from PIL import Image
import os

# Set the input and output directories
input_dir = r"\\shelter\Kyu\unstain2stain\tiles\registered_tiles\HE_train_val_test_split\fold_B\train"
output_dir = r"\\shelter\Kyu\unstain2stain\tiles\registered_tiles\HE_train_val_test_split\fold_B\train\tile_512"

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Loop through all the PNG files in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith(".png"):
        # Open the image
        image = Image.open(os.path.join(input_dir, filename))

        # Get the width and height of the image
        width, height = image.size

        # Check that the image is 1024x1024 pixels
        if width == 1024 and height == 1024:
            # Tile the image into 16 256x256 tiles
            for i in range(2):
                for j in range(2):
                    x = i * 512
                    y = j * 512
                    tile = image.crop((x, y, x + 512, y + 512))

                    # Save the tile with a unique name
                    tile_filename = f"{os.path.splitext(filename)[0]}_{i}_{j}.png"
                    tile.save(os.path.join(output_dir, tile_filename))
